In [23]:
import pandas as pd


df = pd.read_json('data/bloomberg_quint_news.json')

df

,url,title,short_description,author,date_created,date_modified,category,raw_description,description,publisher,scraped_at
0,https://www.bloombergquint.com/markets/all-you...,All You Need To Know Going Into Trade On Septe...,"Stocks in the news, big brokerage calls of the...",Darshan A Nakhwa,"23 Sep 2021, 7:05 AM IST","23 Sep 2021, 7:05 AM IST",Markets,"<div class=""story-element story-element-text"">...",Asian stocks were steady early Thursday after ...,https://www.bloombergquint.com/,2021-09-24 00:01:25
1,https://www.bloombergquint.com/business/bridge...,"Bridgestone CEO Backs Safe Tokyo Olympics, Dia...","Bridgestone CEO Backs Safe Tokyo Olympics, Dia...",Shiho Takezawa &,"23 Apr 2021, 5:35 AM IST","23 Apr 2021, 6:35 AM IST",Business,"<div class=""story-element story-element-text"">...",Bridgestone Corp. will support the Tokyo Olym...,https://www.bloombergquint.com/,2021-09-24 00:01:26
2,https://www.bloombergquint.com/markets/stocks-...,"Stocks To Watch: HCL Tech, Cyient, M&M Financi...",Here are the stocks to watch in trade today...,BQ Desk,"23 Apr 2021, 7:29 AM IST","23 Apr 2021, 7:29 AM IST",Markets,"<div class=""story-element story-element-text"">...",Indian equity benchmarks reversed losses made ...,https://www.bloombergquint.com/,2021-09-24 00:01:26
3,https://www.bloombergquint.com/research-report...,Localised Lockdowns Cannot But Impinge On Econ...,Localised Lockdowns Cannot But Impinge On Econ...,Nirmal Bang Institutional Research,"26 Apr 2021, 7:58 AM IST","26 Apr 2021, 7:58 AM IST",Research Reports,"<div class=""story-element story-element-text"">...","Nirmal Bang Report, We assess the ‘state of af...",https://www.bloombergquint.com/,2021-09-24 00:01:26
4,https://www.bloombergquint.com/business/cp-rai...,CP Rail Wins Regulator Exemption From Tougher ...,CP Rail Wins Regulator Exemption From Tougher ...,Thomas Black,"24 Apr 2021, 7:08 AM IST","24 Apr 2021, 7:44 AM IST",Business,"<div class=""story-element story-element-text"">...",Canadian Pacific Railway Ltd. won a petition ...,https://www.bloombergquint.com/,2021-09-24 00:01:27
...,...,...,...,...,...,...,...,...,...,...,...
7013,https://www.bloombergquint.com/videos/2016/04/...,What's Behind Canada's Employment Surge?,What's Behind Canada's Employment Surge?,None,None,None,Videos,"<div class=""story-element story-element-text"">...",,https://www.bloombergquint.com/,2021-09-24 01:22:29
7014,https://www.bloombergquint.com/videos/2016/04/...,Weeks of Ups and Downs for the S&P 500,Weeks of Ups and Downs for the S&P 500,None,None,None,Videos,"<div class=""story-element story-element-text"">...",,https://www.bloombergquint.com/,2021-09-24 01:22:29
7015,https://www.bloombergquint.com/videos/2016/04/...,Inside the Lowered Expectations for Corporate ...,Inside the Lowered Expectations for Corporate ...,None,None,None,Videos,"<div class=""story-element story-element-text"">...",,https://www.bloombergquint.com/,2021-09-24 01:22:29
7016,https://www.bloombergquint.com/videos/2016/04/...,Who Will Be Peru’s Next President?,Who Will Be Peru’s Next President?,None,None,None,Videos,"<div class=""story-element story-element-text"">...",,https://www.bloombergquint.com/,2021-09-24 01:22:30


In [24]:
def check_missing_values(df):
    # Check for NaN, missing, or NaT values
    missing_rows = df[df.isnull().any(axis=1)]

    if not missing_rows.empty:
        print("Deleting rows with missing values...")
        df.dropna(inplace=True)
        print("Rows with missing values have been deleted.")
    else:
        print("There are no missing values in the DataFrame.")

# Example usage:
check_missing_values(df)


Deleting rows with missing values...
Rows with missing values have been deleted.


In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

def top_10_articles_with_keyword(df, keyword, title_weight=2):
    # Combine title, short description, and description into a single column
    df['combined_text'] = df['title'] + ' ' + df['short_description'] + ' ' + df['description']

    # Initialize TF-IDF vectorizer
    vectorizer = TfidfVectorizer()

    # Fit and transform the combined text using TF-IDF vectorizer
    tfidf_matrix = vectorizer.fit_transform(df['combined_text'])

    # Get the feature names (words)
    feature_names = vectorizer.get_feature_names_out()

    # Convert feature names to a numpy array for easier manipulation
    feature_names_array = np.array(feature_names)

    # Get the indices of the articles containing the keyword (case-insensitive)
    article_indices = np.where(np.any([df['title'].str.contains(keyword, case=False),
                                       df['short_description'].str.contains(keyword, case=False),
                                       df['description'].str.contains(keyword, case=False)], axis=0))[0]

    if len(article_indices) == 0:
        print(f"No articles found containing the keyword '{keyword}'")
        return None

    # Calculate the TF-IDF scores for the articles containing the keyword
    tfidf_scores = tfidf_matrix[article_indices].toarray()

    # Scale TF-IDF scores for the keyword in the title by title_weight
    title_indices = feature_names_array.tolist().index(keyword)
    tfidf_scores[:, title_indices] *= title_weight

    # Calculate the maximum TF-IDF scores for each article
    max_tfidf_scores = tfidf_scores.max(axis=1)

    # get the length of the article_indices
    length = len(article_indices)

    # Get the indices of the top 10 articles with the highest maximum TF-IDF scores
    top_10_indices = np.argsort(max_tfidf_scores)[::-1][:min(len(article_indices), 10)]

    # Get the top 10 articles
    top_10_articles = df.iloc[article_indices[top_10_indices]]

    return top_10_articles, length

# Example usage:
top_10_articles, l = top_10_articles_with_keyword(df, 'france', title_weight=5)
print(l)
if top_10_articles is not None:
    print(top_10_articles)


179
                                                    url  \
1707  https://www.bloombergquint.com/politics/macron...   
474   https://www.bloombergquint.com/business/air-fr...   
3441  https://www.bloombergquint.com/onweb/france-le...   
3261  https://www.bloombergquint.com/business/france...   
3871  https://www.bloombergquint.com/politics/macron...   
3133  https://www.bloombergquint.com/politics/france...   
2302  https://www.bloombergquint.com/onweb/macron-ou...   
1665  https://www.bloombergquint.com/onweb/france-to...   
1244  https://www.bloombergquint.com/business/france...   
2830  https://www.bloombergquint.com/onweb/france-to...   

                                                  title  \
1707  Macron Bets Vaccines Will Kick In Soon in Rush...   
474   Air France-KLM Weighs Fresh Fundraising as Los...   
3441  France Leads U.S., U.K. With Citizens Saying E...   
3261  France Plans Talks on U.S. Flights as It Trial...   
3871  France Backs Chad Military Rulers, AU Urges S

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

def top_10_articles_with_keyword(df, keyword, title_weight=2, short_description_weight=1.5, description_weight=1, sort_by_recent=False):
    # Combine title, short description, and description into a single column
    df['combined_text'] = df['title'] + ' ' + df['short_description'] + ' ' + df['description']

    # Initialize TF-IDF vectorizer
    vectorizer = TfidfVectorizer()

    # Fit and transform the combined text using TF-IDF vectorizer
    tfidf_matrix = vectorizer.fit_transform(df['combined_text'])

    # Get the feature names (words)
    feature_names = vectorizer.get_feature_names_out()

    # Convert feature names to a numpy array for easier manipulation
    feature_names_array = np.array(feature_names)

    # Get the indices of the articles containing the keyword (case-insensitive)
    article_indices = np.where(np.any([df['title'].str.contains(keyword, case=False),
                                       df['short_description'].str.contains(keyword, case=False),
                                       df['description'].str.contains(keyword, case=False)], axis=0))[0]

    if len(article_indices) == 0:
        print(f"No articles found containing the keyword '{keyword}'")
        return None

    # Calculate the TF-IDF scores for the articles containing the keyword
    tfidf_scores = tfidf_matrix[article_indices].toarray()

    # Scale TF-IDF scores for the keyword in the title by title_weight
    title_indices = feature_names_array.tolist().index(keyword)
    tfidf_scores[:, title_indices] *= title_weight

    # Scale TF-IDF scores for the keyword in the short description by short_description_weight
    short_description_indices = feature_names_array.tolist().index(keyword)
    tfidf_scores[:, short_description_indices] *= short_description_weight

    # Scale TF-IDF scores for the keyword in the description by description_weight
    description_indices = feature_names_array.tolist().index(keyword)
    tfidf_scores[:, description_indices] *= description_weight

    # Calculate the maximum TF-IDF scores for each article
    max_tfidf_scores = tfidf_scores.max(axis=1)

    # Get the indices of the top 10 articles with the highest maximum TF-IDF scores
    top_10_indices = np.argsort(max_tfidf_scores)[::-1][:min(len(article_indices), 10)]

    # Get the top 10 articles
    top_10_articles = df.iloc[article_indices[top_10_indices]]

    # Sort the top 10 articles by date_created if sort_by_recent is True
    if sort_by_recent:
        top_10_articles = top_10_articles.sort_values(by='date_created', ascending=False)

    return top_10_articles

# Example usage:
top_10_articles = top_10_articles_with_keyword(df, 'france', title_weight=5, short_description_weight=3, description_weight=1, sort_by_recent=True)
if top_10_articles is not None:
    print(top_10_articles) 


                                                    url  \
1707  https://www.bloombergquint.com/politics/macron...   
1665  https://www.bloombergquint.com/onweb/france-to...   
2302  https://www.bloombergquint.com/onweb/macron-ou...   
3261  https://www.bloombergquint.com/business/france...   
3133  https://www.bloombergquint.com/politics/france...   
2830  https://www.bloombergquint.com/onweb/france-to...   
3871  https://www.bloombergquint.com/politics/macron...   
3441  https://www.bloombergquint.com/onweb/france-le...   
474   https://www.bloombergquint.com/business/air-fr...   
1244  https://www.bloombergquint.com/business/france...   

                                                  title  \
1707  Macron Bets Vaccines Will Kick In Soon in Rush...   
1665  France to Extend Covid Vaccination to All Adul...   
2302  Macron Outlines French Re-Opening Plan Despite...   
3261  France Plans Talks on U.S. Flights as It Trial...   
3133  France Warns U.K. of ‘Retaliation’ in Post-Bre...